In [1]:
# 🔹 Cell 1 — Imports & Config (Phase-4.5 JOIN)
import json
import torch
import torch.nn as nn
import sys
import os
sys.path.append("..")

from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

from src.utils import (
    tokens_to_ids,
    pad_sequence,
    create_attention_mask,
    get_allowed_tokens
)

from src.vocab import PAD, TOKEN2ID, ID2TOKEN, UNK
from models.sql_transformer import SQLTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ CHECKPOINT PATH
CHECKPOINT_DIR = "notebooks/checkpoints/phase4_5_join"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


In [2]:
#🔹 Cell 2 — Phase-4.5 Config
PHASE3_CKPT = "checkpoints/phase3_model.pt"
PHASE4_5_CKPT = "checkpoints/phase4_5_model.pt"

PHASE4_5_PATH = "../data/sql_ast/phase4.5_join.json"

with open(PHASE4_5_PATH, "r") as f:
    phase4_5_data = json.load(f)

print("Total Phase-4.5 samples:", len(phase4_5_data))


Total Phase-4.5 samples: 2000


In [3]:
#🔹 Cell 3 — Decoder Training Sample Builder (UNCHANGED LOGIC)
def prepare_phase4_5_sample(sample):
    """
    Converts phase4.5_joins.json entry into decoder input / label pairs.
    """

    tokens = sample["input_tokens"]

    token_ids = [
        TOKEN2ID.get(t, TOKEN2ID[UNK])
        for t in tokens
    ]

    input_ids = torch.tensor(token_ids[:-1], dtype=torch.long)
    labels = torch.tensor(token_ids[1:], dtype=torch.long)

    labels[labels == TOKEN2ID[PAD]] = -100

    return {
        "input_ids": input_ids,
        "labels": labels
    }


In [4]:
# 🔹 Cell 4 — Dataset
class Phase45JoinDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return prepare_phase4_5_sample(self.data[idx])


In [5]:
# 🔹 Cell 5 — Collate Function (PAD + MASK)
def phase4_5_collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = [item["labels"] for item in batch]

    max_len = max(x.size(0) for x in input_ids)

    padded_inputs, padded_labels = [], []

    for inp, lab in zip(input_ids, labels):
        pad_len = max_len - inp.size(0)

        padded_inputs.append(
            torch.cat([
                inp,
                torch.full((pad_len,), TOKEN2ID[PAD], dtype=torch.long)
            ])
        )

        padded_labels.append(
            torch.cat([
                lab,
                torch.full((pad_len,), -100, dtype=torch.long)
            ])
        )

    return {
        "input_ids": torch.stack(padded_inputs),
        "labels": torch.stack(padded_labels)
    }


In [6]:
# 🔹 Cell 6 — DataLoader
BATCH_SIZE = 16

train_dataset = Phase45JoinDataset(phase4_5_data)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=phase4_5_collate_fn
)


In [7]:
#🔹 Cell 7 — Grammar-Masked Loss (NO LOGIC CHANGE)
import torch.nn.functional as F

def phase4_5_loss(logits, input_ids, labels, allowed_token_fn):
    B, T, V = logits.size()
    mask = torch.zeros((B, T, V), device=logits.device)

    for b in range(B):
        for t in range(T):
            if labels[b, t] == -100:
                continue

            ids = input_ids[b, :t+1].tolist()
            tokens_so_far = [ID2TOKEN.get(i, UNK) for i in ids]

            allowed = allowed_token_fn(
                tokens_so_far=tokens_so_far,
                schema_tables=None,
                schema_columns=None,
                intent_signals=None   # ✅ SAFE FOR PHASE-4.5
            )

            if allowed:
                m = torch.full((V,), -500.0, device=logits.device)
                for tid in allowed:
                    if tid < V:
                        m[tid] = 0.0
                mask[b, t, :] = m

    masked_logits = logits + mask
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    return loss_fn(masked_logits.view(-1, V), labels.view(-1))


In [8]:
# 🔹 Cell 8 — Metrics (UNCHANGED)
def compute_prf(preds, labels, ignore_index=-100):
    preds = preds.view(-1)
    labels = labels.view(-1)

    mask = labels != ignore_index
    preds = preds[mask]
    labels = labels[mask]

    tp = (preds == labels).sum().item()
    fp = (preds != labels).sum().item()
    fn = fp

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)

    return precision, recall, f1


In [9]:
# 🔹 Cell 9 — Load Phase-3 Model (VOCAB SURGERY SAFE)
model = SQLTransformer().to(device)

checkpoint_state = torch.load(PHASE3_CKPT, map_location=device)
model_state = model.state_dict()

mismatched = ["embedding.weight", "fc_out.weight", "fc_out.bias"]

for name, param in checkpoint_state.items():
    if name in mismatched:
        new_param = model_state[name]
        if param.ndim > 1:
            new_param[:param.size(0), :] = param
        else:
            new_param[:param.size(0)] = param
        model_state[name] = new_param
    else:
        model_state[name] = param

model.load_state_dict(model_state)

print("✅ Phase-3 → Phase-4.5 vocab surgery complete")


✅ Phase-3 → Phase-4.5 vocab surgery complete


In [10]:
#🔹 Cell 10 — Grammar Sanity Check (OPTIONAL BUT SAFE)
def verify_grammar_with_data(dataset, allowed_fn):
    conflicts = 0
    skipped = 0

    for sample in dataset:
        tokens = sample["input_tokens"]

        for t in range(len(tokens) - 1):
            next_tok = tokens[t + 1]

            # 🔥 SKIP tokens that are NOT part of model vocab
            if next_tok not in TOKEN2ID:
                skipped += 1
                continue

            allowed = allowed_fn(
                tokens_so_far=tokens[:t+1],
                schema_tables=None,
                schema_columns=None,
                intent_signals=None
            )

            if TOKEN2ID[next_tok] not in allowed:
                conflicts += 1

    print(f"Grammar conflicts: {conflicts}")
    print(f"Skipped non-learnable tokens: {skipped}")



In [11]:
# 🔹 Cell 11 — Freeze Strategy
for name, param in model.named_parameters():
    if "layers.1" in name or "layers.2" in name or "fc_out" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [12]:
#🔹 Cell 12 — Optimizer
optimizer = AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=3e-4
)


In [13]:
# 🔹 Cell 13 — Training Loop (Phase-4.5 with Checkpoints)
EPOCHS = 30
model.train()

best_loss = float("inf")  # 🔥 Track best model

for epoch in range(EPOCHS):
    total_loss, steps = 0.0, 0
    total_p, total_f1 = 0.0, 0.0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids)
        loss = phase4_5_loss(logits, input_ids, labels, get_allowed_tokens)

        # Safety check
        if torch.isnan(loss):
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Metrics
        preds = torch.argmax(logits, dim=-1)
        p, _, f1 = compute_prf(preds, labels)

        total_loss += loss.item()
        total_p += p
        total_f1 += f1
        steps += 1

    # ----------------------------
    # Epoch summary
    # ----------------------------
    avg_loss = total_loss / steps
    avg_p = total_p / steps
    avg_f1 = total_f1 / steps

    print(
        f"Epoch {epoch+1:02d} | "
        f"Loss: {avg_loss:.4f} | "
        f"P: {avg_p:.4f} | "
        f"F1: {avg_f1:.4f}"
    )

    # ----------------------------
    # 🔥 SAVE CHECKPOINT (EVERY EPOCH)
    # ----------------------------
    epoch_ckpt_path = os.path.join(
        CHECKPOINT_DIR,
        f"phase4_5_epoch_{epoch+1:02d}.pt"
    )

    torch.save(
        {
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "avg_loss": avg_loss
        },
        epoch_ckpt_path
    )

    print(f"💾 Saved epoch checkpoint: {epoch_ckpt_path}")

    # ----------------------------
    # 🔥 SAVE BEST MODEL
    # ----------------------------
    if avg_loss < best_loss:
        best_loss = avg_loss

        best_ckpt_path = os.path.join(
            CHECKPOINT_DIR,
            "phase4_5_best.pt"
        )

        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_loss": best_loss
            },
            best_ckpt_path
        )

        print(f"🏆 New best model saved (loss={best_loss:.4f})")


Epoch 01 | Loss: 89.1644 | P: 0.6132 | F1: 0.6132
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_01.pt
🏆 New best model saved (loss=89.1644)
Epoch 02 | Loss: 87.3003 | P: 0.7056 | F1: 0.7056
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_02.pt
🏆 New best model saved (loss=87.3003)
Epoch 03 | Loss: 85.6037 | P: 0.6432 | F1: 0.6432
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_03.pt
🏆 New best model saved (loss=85.6037)
Epoch 04 | Loss: 83.8162 | P: 0.6025 | F1: 0.6025
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_04.pt
🏆 New best model saved (loss=83.8162)
Epoch 05 | Loss: 82.0875 | P: 0.5609 | F1: 0.5609
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_05.pt
🏆 New best model saved (loss=82.0875)
Epoch 06 | Loss: 80.3018 | P: 0.5272 | F1: 0.5272
💾 Saved epoch checkpoint: notebooks/checkpoints/phase4_5_join\phase4_5_epoch_06.pt
🏆 New best m